In [ ]:
#In this lab, you will be using the [Sakila](https://dev.mysql.com/doc/sakila/en/) database of movie rentals.

In [ ]:
#In order to optimize our inventory, we would like to predict if a film will have more monthly rentals in July than in June. Create a model to predict it.

In [5]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass() 

 ········


In [6]:
import numpy as np 

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005

In [8]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

query = ''' SELECT f.film_id, f.title, f.length, f.rental_rate, f.release_year, f_a.actor_id, CONCAT(a.first_name, " ",a.last_name) as actor_name, f_c.category_id, c.name as category
FROM film as f 
JOIN film_category as f_c 
ON f.film_id = f_c.film_id 
JOIN category as c 
ON f_c.category_id = c.category_id 
JOIN film_actor as f_a 
ON f.film_id = f_a.film_id 
JOIN actor as a 
ON f_a.actor_id = a.actor_id
ORDER BY f.film_id ASC '''

data = pd.read_sql_query(query, engine)
data.head()

AttributeError: 'OptionEngine' object has no attribute 'execute'

In [ ]:
#2. Create a query to get the total amount of rentals in June for each film. 

In [9]:
query = '''
SELECT f.film_id, f.title, COUNT(p.rental_id) as num_of_rentals
FROM film as f
JOIN inventory as i 
ON f.film_id = i.film_id 
JOIN rental as r 
ON i.inventory_id = r.inventory_id 
JOIN payment as p 
ON r.rental_id = p.rental_id
WHERE (rental_date BETWEEN "2005-06-01 00:00:00" AND "2005-06-30 23:59:59")
GROUP BY f.film_id
ORDER BY COUNT(p.rental_id) DESC;
'''

data2 = pd.read_sql_query(query, engine)
data2.head()


AttributeError: 'OptionEngine' object has no attribute 'execute'

In [ ]:
#3. Do the same with July. 

In [10]:
query = '''
SELECT f.film_id, f.title, COUNT(p.rental_id) as num_of_rentals
FROM film as f
JOIN inventory as i 
ON f.film_id = i.film_id 
JOIN rental as r 
ON i.inventory_id = r.inventory_id 
JOIN payment as p 
ON r.rental_id = p.rental_id
WHERE (rental_date BETWEEN "2005-07-01 00:00:00" AND "2005-07-31 23:59:59")
GROUP BY f.film_id
ORDER BY COUNT(p.rental_id) DESC;
'''

data3 = pd.read_sql_query(query, engine)
data3.head()

AttributeError: 'OptionEngine' object has no attribute 'execute'

In [ ]:
#4. Create a new column containing (Yes/No) for each film whether or not the number of monthly rentals in **July was bigger than in June**. Your objective will be to predict this new column.

In [ ]:
query = ''' 
SELECT f.film_id, f.title, 
       COUNT(DISTINCT p.rental_id) as num_of_rentals,
       (CASE WHEN 
           (SUM(CASE WHEN rental_date BETWEEN "2005-07-01 00:00:00" AND "2005-07-31 23:59:59" THEN 1 ELSE 0 END)
            > 
            SUM(CASE WHEN rental_date BETWEEN "2005-06-01 00:00:00" AND "2005-06-30 23:59:59" THEN 1 ELSE 0 END)) 
        THEN "Yes" ELSE "No" END) as more_rentals_in_july_than_june
FROM film as f
JOIN inventory as i
ON f.film_id = i.film_id 
JOIN rental as r
ON i.inventory_id = r.inventory_id 
JOIN payment as p
ON r.rental_id = p.rental_id 
GROUP BY f.film_id, f.title
ORDER BY SUM(CASE WHEN rental_date BETWEEN "2005-07-01 00:00:00" AND "2005-07-31 23:59:59" THEN 1 ELSE 0 END) DESC;
'''

data4 = pd.read_sql_query(query, engine) 
data4.head()

In [ ]:
#5. Read the data into a Pandas dataframe.


In [ ]:
df1 = pd.read_sql('SELECT * FROM june_july_predictions', engine)

In [ ]:
df = pd.DataFrame(df1) 

df.head()

In [ ]:
#7. Analyze extracted features and transform them. You may need to encode some categorical variables or scale numerical variables.

In [ ]:
df_numeric = df.select_dtypes(include = np.number)
df_categorical = df.select_dtypes("object")

In [ ]:
df_numeric.info()

In [ ]:
df_categorical.info()

In [ ]:
df_numeric.isna().sum()

In [ ]:
df_categorical.isna().sum()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoder.fit(df_categorical)

categorical_encoded = encoder.transform(df_categorical).toarray()

df_categorical_encoded = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out())

In [ ]:
new_df = pd.concat([df_numeric, df_categorical_encoded], axis = 1)

In [ ]:
#8. Create a logistic regression model to predict this new column from the cleaned data.

In [ ]:
#Order of Operations- 
# 1. X-y split- isolate the column we want to predict
# 2. Train-test split- establish X_train/test, y_train/test
# 3. Introduce a scaler to ensure all of our numerical variables are on the same scale- use MinMax because we're using the encoder next
# 4. Introduce a OneHot Encoder to encode all of the categorical variables  
# 5. Concat our encoded with the scaled numerical variables produced in Step 3 
# 6. Use model.fit() to "train" the model 

# 7. For the numericals, do logistic regression- lr = LogisticRegression(random_state = 0, solver = 'lbfgs'), lr.fit(X_train_scaled, y_train)
# 8. Compute the r2score, mae, mse, rmse for the numericals as well
# 8. Start getting probabilities, apply lr.predict_proba(X_train_scaled).sum(axis = 1), apply later lr.predict(X_train_scaled) 
# 9. Compute the logarithim- logits 
# 10. Display and visualise using sns.scatterplot(y = logits, x = X_train_scaled['Column']); plt.show() 
# 11. For the categoricals, calculate accuracy, precision, recall, F1, Cohen-Kappa 
# 12. Generate a confusion matrix 


In [ ]:
# 1. X-y split- isolate the column we want to predict- I'm not sure which we're supposed to predict from above

In [ ]:
y = new_df['in_the_top_20_actors']
X = new_df.drop(['in_the_top_20_actors'], axis=1)

In [ ]:
# 2. Train-test split- establish X_train/test, y_train/test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [ ]:
# 3. Introduce a scaler to ensure all of our numerical variables are on the same scale- use MinMax because we're using the encoder next

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

X_train_num = X_train.select_dtypes(include = np.number)
X_test_num = X_test.select_dtypes(include = np.number)

transformer = MinMaxScaler().fit(X_train_num) 
X_train_normalized = transformer.transform(X_train_num)
X_test_normalized = transformer.transform(X_test_num)

X_train_norm = pd.DataFrame(X_train_normalized, columns = X_train_num.columns)
X_test_norm = pd.DataFrame(X_test_normalized, columns = X_test_num.columns)

In [ ]:
X_train_norm.columns = X_train_num.columns
X_train_norm.head()

In [ ]:
# 5. Concat our encoded with the scaled numerical variables produced in Step 3 

In [ ]:
df_scaled_encoded = pd.concat([X_train_norm, df_categorical_encoded], axis=1)

In [ ]:
# 6. Use model.fit() to "train" the model 

In [ ]:
model = df_scaled_encoded()
model.fit()

In [ ]:
# 7. For the numericals, do logistic regression- lr = LogisticRegression(random_state = 0, solver = 'lbfgs'), lr.fit(X_train_scaled, y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(random_state = 0, solver = 'lbfgs') 
lr.fit(X_train_scaled, y_train)

In [ ]:
# 8. Compute the r2score, mae, mse, rmse for the numericals as well

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_squared_error

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

In [ ]:
# 8. Start getting probabilities, apply lr.predict_proba(X_train_scaled).sum(axis = 1), apply later lr.predict(X_train_scaled) 

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)

In [ ]:
# 11. For the categoricals, calculate accuracy, precision, recall, F1, Cohen-Kappa 

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
f1 = metrics.f1_score(y_test, y_pred)

In [ ]:
# 12. Generate a confusion matrix 

In [ ]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y_test, y_pred)
print(matrix)